### 파이토치 구성요소  
- torch : 메인, 텐서 등의 다양한 수학 함수 포함  
- torch.autograd : 자동 미분 기능을 제공  
- torch.nn : 신경망 구축을 위한 데이터 구조나 레이어  
- torch.multiprocessing : 병렬처리 기능을 제공  
- torch.optim : SGD(Stochastic Gradient Descent(확률적 경사 하강법))를 중심으로 한 파라미터 최적화 알고리즘 제공  
- torch.utils : 데이터 조작 등 유틸리티 기능 제공  
- torch.onnx : ONNX(Open Neural Network Exchange) 서로 다른 프레임워크 간의 모델을 공유할 때 사용

In [7]:
import torch

x = torch.empty(4,2)
y = torch.rand(4,2)
z = torch.zeros(4,2, dtype=torch.long)
print(x)
print(y)
print(z)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[0.8722, 0.2288],
        [0.7902, 0.9704],
        [0.8743, 0.7566],
        [0.5938, 0.1040]])
tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0]])


In [10]:
# 사용자 입력 텐서
x = torch.tensor([3, 2.3])
y = torch.ones(2,4, dtype=torch.double)
y

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=torch.float64)

### view : 텐서의 크기(size)나 모양(shape)을 변경  
- 기본적으로 변경 전/후 원소 개수 유지되야함  
- -1로 설정하면 자동으로 지정  
### item : 텐서에 값이 하나만 존재할 경우 숫자값 반환  
### squeeze : 차원 축소  
### unsqueeze : 차원 확대 

In [24]:
x = torch.rand(1,3,3)
print(x)
print(x.view(-1))
print(x.squeeze())

y = torch.rand(3,3)
print(y)
print(y.unsqueeze(dim=0))

tensor([[[0.6024, 0.1715, 0.4026],
         [0.1609, 0.8307, 0.0230],
         [0.9170, 0.1547, 0.2351]]])
tensor([0.6024, 0.1715, 0.4026, 0.1609, 0.8307, 0.0230, 0.9170, 0.1547, 0.2351])
tensor([[0.6024, 0.1715, 0.4026],
        [0.1609, 0.8307, 0.0230],
        [0.9170, 0.1547, 0.2351]])
tensor([[0.0692, 0.3262, 0.3707],
        [0.8867, 0.4486, 0.7824],
        [0.9856, 0.0884, 0.2417]])
tensor([[[0.0692, 0.3262, 0.3707],
         [0.8867, 0.4486, 0.7824],
         [0.9856, 0.0884, 0.2417]]])


## Autograd(자동미분)  
- torch.autograd는 Tensor의 모든 연산에 대해 자동 미분 제공  
- 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻  
- backprop(역전파)를 위해 미분값을 자동으로 계산  

requires_grad 속성을 True로 설정하면, 해당 텐서에서 이루어지는 모든 연산들을 추적하기 시작  
기록을 추적하는 것을 중단하게하려면, .detach() 를 호출

In [26]:
a = torch.randn(3,3)
a = a*3
print(a)
print(a.requires_grad) #기본값은 False

tensor([[ 1.3290, -3.0242, -1.7909],
        [-0.8860,  3.1785,  0.8304],
        [ 3.6662,  4.6558, -2.2917]])
False


In [ ]:
a.requires_grad_(True) #in-place 

In [27]:
x = torch.ones(3, 3, requires_grad=True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


In [28]:
y = x+5
print(y)

tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]], grad_fn=<AddBackward0>)


계산이 완료된 후, .backward()를 호출하면 자동으로 역전파 계산한 다음에 .grad속성에 누적함

In [34]:
z = y.sum()
print(z)
z.backward()

tensor(54., grad_fn=<SumBackward0>)


grad : data가 거쳐온 layer에 대한 미분값 저장

In [36]:
print(x)
print(x.grad)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[1.4444, 1.4444, 1.4444],
        [1.4444, 1.4444, 1.4444],
        [1.4444, 1.4444, 1.4444]])


#### 자동미분 흐름 예시  
- 계산 흐름 : a -> b -> c -> out  
- 미분 out / a = ?  
- backward()를 통해 a <- b <- c <- out을 계산하면 미분 out / a 값이 a.grad에 채워짐

In [38]:
a = torch.ones(2,2,requires_grad=True)
print(a)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [41]:
print(a.data) 
print(a.grad) #초기
print(a.grad_fn)

tensor([[1., 1.],
        [1., 1.]])
None
None


In [42]:
b = a + 2
print(b)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [43]:
c = b ** 2
print(c)

tensor([[9., 9.],
        [9., 9.]], grad_fn=<PowBackward0>)


In [44]:
out = c.sum()
print(out)

tensor(36., grad_fn=<SumBackward0>)


In [45]:
out.backward()

tensor(36., grad_fn=<SumBackward0>)


In [46]:
print(a.data) 
print(a.grad) # 왜 값이 6이나왔냐면....
print(a.grad_fn) #직접 계산한게 없어서 None

tensor([[1., 1.],
        [1., 1.]])
tensor([[6., 6.],
        [6., 6.]])
None


In [47]:
print(b.data) 
print(b.grad) 
print(b.grad_fn)

tensor([[3., 3.],
        [3., 3.]])
None


C:\Users\JangChanYoung\AppData\Local\Temp\ipykernel_38456\3135712180.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print(b.grad)


In [48]:
print(c.data) 
print(c.grad) 
print(c.grad_fn)

tensor([[9., 9.],
        [9., 9.]])
None


C:\Users\JangChanYoung\AppData\Local\Temp\ipykernel_38456\823509336.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print(c.grad)


In [49]:
print(out.data) 
print(out.grad) 
print(out.grad_fn)

tensor(36.)
None


C:\Users\JangChanYoung\AppData\Local\Temp\ipykernel_38456\3004614393.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print(out.grad)


파이터치에서는 데이터준비를 위해 torch.utils.data의 Dataset과 DataLoader 사용가능  
 - Dataset에는 다양한 데이터셋이 존재 (ex MNIST)  
 - DataLoader와 Dataset을 통해 batch_size, train여부, transform등을 인자로 넣어 데이터를 어떻게 load할 것인지 정해줄 수 있음

In [50]:
from torch.utils.data import Dataset, DataLoader

토치비전(torchvision)은 파이토치에서 제공하는 데이터셋들이 모여있는 패키지  
 - transforms : 전처리할 떄 사용하는 메소드  
 - transforms : 에서 제공하는 클래스 이외는 일반적으로 클래스를 따로 만들어 전처리 단계 진행

In [ ]:
import torchvision.transforms as transforms
from torchvision import datasets